# Pre-Process and Load Microsoft Academic Graph

## Preprocess MAG

### Libraries

In [ ]:
## Basic
import numpy as np
from collections import defaultdict
import operator
import pandas as pd 
import random
import time


## Visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Network Processing

## networkx
import networkx as nx
from networkx.generators import random_graphs
from networkx.generators import social
from networkx.generators import classic


## smallworld
from smallworld.draw import draw_network
from smallworld import get_smallworld_graph

## snap
color_map =["grey"]

## graph sampling
from sampling import ForestFire, Metropolis_Hastings, Random_Walk, Snowball, Ties, Base_Samplers

## Preprocess Files

In [ ]:
f = open('data/mag/mag_raw.txt','r') 

L = list()
A = defaultdict(int) # default value of int is 0

line = f.readline()

n_list = list()

while line:
    
    if "#@" in line:
        n_list = line.replace("\n", "").split("#@")[1].split(",")
        
        ## edges
        if len(n_list) > 1: 
            for i in range(0, len(n_list)):
                for j in range(i+1, len(n_list)):
                        L.append((n_list[i], n_list[j]))                
        else:
            L.append((n_list[0], n_list[0]))
            
            
                        
    if "#citation" in line:
        
        c = line.split("#citation")[1]
        c = abs(int(c))
        
        for n in n_list:
            A[n] +=  c
                
            #print("n", n, "c", A[n], "\n")
    
    line = f.readline() ## read next line

f.close()

## pre-process and sort
L = set(L)

A_sorted = sorted(A.items(), key=operator.itemgetter(1), reverse=True)
A_sorted = [n_a[0] for n_a in A_sorted]

In [ ]:
print(len(L))
print(list(A.keys())[:30])
print(A["Leslie Lamport"])
print(A_sorted[2])
print(len(A_sorted))

### Numberize

In [ ]:
Num = list(range(0, len(A_sorted)))
num_author = dict(zip(Num, A_sorted))
author_num = dict(zip(A_sorted, Num))

In [ ]:
## Links
L_num = list()
Source_num = list()
Target_num = list()

for l in L:
    L_num.append((author_num[l[0]], author_num[l[1]]))
    Source_num.append(author_num[l[0]])
    Target_num.append(author_num[l[1]])

In [ ]:
## Citations
C_num = list()

for num in Num:
    c = A[num_author[num]]
    C_num.append(c)
    
## Names
Names_num = list()

for num in Num:
    name = num_author[num]
    Names_num.append(name)

### Save as CSV

In [ ]:
nodes = pd.DataFrame(
    {'Id': Num,
     'name': Names_num,
     'citations': C_num,
    })

edges = pd.DataFrame(
    {'Source': Source_num,
     'Target': Target_num,
    })

nodes.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/nodes_py.csv", index = None, header=False)
edges.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/edges_py.csv", index = None, header=False)

In [ ]:
nodes.groupby('citations').count()

## Sample Subset Fix

In [ ]:
df_edges = pd.read_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/edges.csv")
df_nodes  = pd.read_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/nodes.csv")

edges = df_edges.values
nodes = df_nodes.values

#df_edges.columns.values
#df_nodes.columns.values

In [ ]:
edges = [l.tolist() for l in list(edges)]
nodes = [l.tolist() for l in list(nodes)]

In [ ]:
nodes = [i[0] for i in nodes]

In [ ]:
Sample_nodes = random.sample(nodes, 200)
Sample_edges = list()

for n1,n2 in edges:
    if n1 in Sample_nodes and n2 in Sample_nodes:
        
        Sample_edges.append([n1,n2])

In [ ]:
#Sample_edges = random.sample(nodes, 200)
#Sample_nodes = list()

#Sample_edges_flattened = [val for sublist in Sample_edges for val in sublist]

#for num, name, c in nodes:
#    if num in Sample_edges_flattened:
        
#        Sample_nodes.append(num)
    

## Save Samples

In [ ]:
## Citations
C_num = list()

for num in Sample_nodes:
    c = A[num_author[num]]
    C_num.append(c)
    
## Names
Names_num = list()

for num in Sample_nodes:
    name = num_author[num]
    Names_num.append(name)
    
    
## Links
L_num = list()
Source_num = list()
Target_num = list()

for l in Sample_edges:
    L_num.append((l[0],l[1]))
    Source_num.append(l[0])
    Target_num.append(l[1])

In [ ]:
nodes = pd.DataFrame(
    {'Id': Sample_nodes,
     'name': Names_num,
     'citations': C_num,
    })

edges = pd.DataFrame(
    {'Source': Source_num,
     'Target': Target_num,
    })

nodes.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/sample_nodes.csv", index = None, header=True)
edges.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/sample_edges.csv", index = None, header=True)

## Sample with Biased Random Walk

In [ ]:
g_complete = nx.read_edgelist("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/edges_py.csv",  nodetype=int, delimiter = ",")
#a_complete = nx.adjacency_matrix(g_complete)

df_nodes  = pd.read_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/nodes_py.csv", header = None)
nodes = df_nodes.values
nodes = [l.tolist() for l in list(nodes)]
nodes_num = [i[0] for i in nodes]

g.add_nodes_from(nodes_num)

In [ ]:
print(len(nodes_num))
print(len(g_complete.edges()))

In [ ]:
node_attr_dict = dict()

for num, name, c in nodes:
    node_attr_dict[num] = c
    
print(node_attr_dict[0])
    
nx.set_node_attributes(g_complete, node_attr_dict, "citations")

In [ ]:
print(len(g_complete.nodes()))
print(list(g_complete.nodes())[:10])

In [ ]:
sampleArgs = {"sample": "biased_random_walk", "jump_bias": "random_walk_induced_graph_sampling", "n": 1000, "p": 20.0, "q": 100.0, "source_starts": 2, "source_returns": 4, "depth": 2}

##exact_n: forestfire, random_walk_induced_graph_sampling, random_walk_sampling_with_fly_back, adjacency, select
##approx_n: snowball, bfs, walk, jump

def get_graph(sampleArgs,g_complete,a_complete):
    
    if sampleArgs["sample"] == "biased_random_walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        #sampler = Base_Samplers(g_complete,a_complete)
        g = sampler.biased_random_walk(sampleArgs["n"], sampleArgs["p"], sampleArgs["q"])

    if sampleArgs["sample"] == "forestfire":
        sampler = ForestFire.ForestFire(g_complete,a_complete)
        g = sampler.forestfire(sampleArgs["n"])

    if sampleArgs["sample"] == "snowball":
        sampler = Snowball.Snowball(g_complete,a_complete)
        g = sampler.snowball(sampleArgs["source_starts"], sampleArgs["source_returns"])

    if sampleArgs["sample"] == "random_walk_induced_graph_sampling":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_induced_graph_sampling(sampleArgs["n"])

    if sampleArgs["sample"] == "random_walk_sampling_with_fly_back":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_sampling_with_fly_back(sampleArgs["n"], sampleArgs["p"])
        
    if sampleArgs["sample"] == "standard_bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.standard_bfs(sampleArgs["source_starts"], sampleArgs["depth"]) 
        
    if sampleArgs["sample"] == "bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.bfs(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.walk(sampleArgs["source_starts"], sampleArgs["source_returns"], sampleArgs["p"])        
        
    if sampleArgs["sample"] == "jump":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.jump(sampleArgs["source_starts"], sampleArgs["p"], sampleArgs["jump_bias"])
        
    if sampleArgs["sample"] == "adjacency":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "select":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
    
    return g 

start_time = time.time()
g = get_graph(sampleArgs, g_complete, a_complete)

print("-- n_max should be >=", len(g), "--")
print("-- function get_graph takes %s secs --" % round((time.time() - start_time),  5))

if len(g) <= 200:
    nx.draw(g, node_color = color_map, with_labels = False)

In [ ]:
Sample_nodes = list(g.nodes())
Sample_edges = list(g.edges())

print(len(Sample_edges))

In [ ]:
## Citations
C_num = list()

for num in Sample_nodes:
    c = A[num_author[num]]
    C_num.append(c)
    
## Names
Names_num = list()

for num in Sample_nodes:
    name = num_author[num]
    Names_num.append(name)
    
    
## Links
L_num = list()
Source_num = list()
Target_num = list()

for l in Sample_edges:
    L_num.append((l[0],l[1]))
    Source_num.append(l[0])
    Target_num.append(l[1])

In [ ]:
nodes = pd.DataFrame(
    {'Id': Sample_nodes,
     'name': Names_num,
     'citations': C_num,
    })

edges = pd.DataFrame(
    {'Source': Source_num,
     'Target': Target_num,
    })

nodes.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/sample_nodes.csv", index = None, header=True)
edges.to_csv("/Users/niklasstoehr/Programming/thesis/4_real_attr/data/mag/sample_edges.csv", index = None, header=True)

### Analyze

In [ ]:
#nx.degree_assortativity_coefficient(g_complete, x='out', y='in')

In [ ]:
from scipy.stats.stats import pearsonr

degrees = list(g_complete.degree())
D = list()
C = list()

for num, degree in degrees:
    c = A[num_author[num]]
    
    D.append(degree)
    C.append(c)

In [ ]:
pearsonr(D, C)